In [1]:
import pandas as pd
import pickle
import numpy as np
import os

from find_rod_mofs.filter_metal_channels.filter_metal_channels import get_channels, \
        get_metallic_substructure

from pymatgen import Structure
import pymatgen
import scipy

In [2]:
df_all = pd.read_pickle('df.pkl')
#df_all = df_all.drop(columns="Unnamed: 0")
df_all = df_all.drop(columns="channel_indices_2")
df_all.head()

,label,normalization_factors,vbm,cbm,homo_adj,lumo_adj,homo_above_vacuum,lumo_above_vacuum,vacuum_potential,band_gap_energy,...,S_in_ligand,ligands,ligand_elements,Num_N_ligands,Num_S_ligands,ligand_atomic_radii,N_in_ligand,channel_indices,distances,alignments
0,xopkuj,0.487495,0.199497,3.683041,-0.500503,4.583041,-5.398656,-0.315112,4.898153,3.483544,...,False,"[([0.61872013 0.80074068 7.49630542] O, (0, 0,...","[O, O, N, N, O, O]",2,0,"[0.6 ang, 0.6 ang, 0.65 ang, 0.65 ang, 0.6 ang...",True,[],[],None
1,yudkir,0.483830,NaN,NaN,NaN,NaN,NaN,NaN,3.513211,NaN,...,False,"[([ 8.74339627 22.37602644 1.17322514] N, (0,...","[N, N, N, N, N, O]",5,0,"[0.65 ang, 0.65 ang, 0.65 ang, 0.65 ang, 0.65 ...",True,"[(0, 0), (1, 1), (2, 2), (3, 3), (6, 6), (7, 7)]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,zejyev,0.294397,0.634390,3.386016,-0.065610,4.286016,-4.264834,0.086792,4.199224,2.751626,...,False,"[([10.9724784 4.9274879 1.24835071] O, (0,...","[O, N, N, O, O]",2,0,"[0.6 ang, 0.65 ang, 0.65 ang, 0.6 ang, 0.6 ang]",True,[],[],None
3,xefbal,0.296614,1.128213,3.511875,0.428213,4.411875,-4.066935,-0.083274,4.495149,2.383662,...,False,"[([4.14079395 0.8506101 7.15618865] O, (0, 0,...","[O, O]",0,0,"[0.6 ang, 0.6 ang]",False,"[(0, 0), (0, 0), (2, 2), (2, 2), (3, 3), (3, 3)]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,witqib,0.514832,0.799362,3.159399,0.099362,4.059399,-5.927756,-1.967719,6.027118,2.360037,...,False,"[([8.44892423 9.47775303 8.04833625] O, (0, 0,...",[O],0,0,[0.6 ang],False,[],[],None


In [17]:
def get_channel_indices(row, white_list=['O','H'],
                        tolerance_factor_1=2.0,
                        tolerance_factor_2=1.0,
                        max_distance_factor=3.0,
                        relaxed=True,
                        other_metals=['Al', 'In', 'Ga']):
    if relaxed:
        s = Structure.from_file(row['relaxed_file'])
    else:
        s = Structure.from_file(row['original_file']) ## need to make this column still!!!!
    ms, non_ms = get_metallic_substructure(s, white_list=white_list, other_metals=other_metals)
    channels = get_channels(ms, non_ms,
                           tolerance_factor_1=tolerance_factor_1,
                           tolerance_factor_2=tolerance_factor_2,
                            max_distance_factor=max_distance_factor)
    return channels


def get_metal_coordinates(row):
    
    s = pymatgen.Structure.from_file(row['relaxed_file'])
    metal_coords = [(i, s.sites[i].coords) for \
              i in range(len(s)) if s.sites[i].specie.is_metal]
    return metal_coords

# def get_distances(row):
    
#     s = pymatgen.Structure.from_file(row['relaxed_file'])
#     dists = []
#     for index_tuple in row['channel_indices']:
        
#         p1 = s[index_tuple[0]].coords
#         p2 = s[index_tuple[1]].coords
#         dist = ( ( p2[0] - p1[0] )**2 + ( p2[1] - p1[1] )**2 + ( p2[2] - p1[2] )**2 )**(1/2)
#         dists.append(dist)
#     return dists


def get_distances(row):
    
    s = pymatgen.Structure.from_file(row['relaxed_file'])
    dists = []
    for index_tuple in row['channel_indices']:
        
        p1 = s[index_tuple[0]].coords
        
        if isinstance(index_tuple[1], int):
            p2 = s[index_tuple[1]].coords
            print(str(p2)+"   1")
        else:
            p2 = p1 + index_tuple[1]#/np.linalg.norm(index_tuple[1])
            print(str(p2)+"   2")
        dist = ( ( p2[0] - p1[0] )**2 + ( p2[1] - p1[1] )**2 + ( p2[2] - p1[2] )**2 )**(1/2)
        dists.append(dist)
    print()
    return dists


def get_alignments(row):
    
    s = pymatgen.Structure.from_file(row['relaxed_file'])
    alignments = []
    for i, channel_1 in enumerate(row['channel_indices']):
        for j, channel_2 in enumerate(row['channel_indices']):
            if i >= j:
                continue
            if channel_1 != channel_2:
                
                p1 = s[channel_1[0]].coords
                p2 = s[channel_1[1]].coords
                
                v1 = (p1 - p2)
                
                p3 = s[channel_2[0]].coords
                p4 = s[channel_2[1]].coords
                
                v2 = p3 - p4
                vec_alignment = np.dot(v1,v2)
                alignments.append(vec_alignment)
    if len(alignments) > 0:
        return alignments
    else:
        return None

In [18]:
a = df_all[:5]
for i in range(len(a)):
    print(get_metal_coordinates(a.iloc[i]))
    print()

[(0, array([0.0098085 , 0.00763981, 5.48387771])), (1, array([ 4.36854295,  5.56587846, -1.38752416])), (2, array([8.7784403 , 0.04506381, 2.72676778])), (3, array([13.13719253,  5.54096603, -4.21056842]))]

[(0, array([ 7.10552056, 21.14404621,  1.67739378])), (1, array([ 0.35602679, 29.51186372,  1.71080216])), (2, array([14.53579404, 12.58900278,  2.43199338])), (3, array([7.77793995, 4.28696734, 2.40905736])), (4, array([ 7.82328613, 12.58900278,  6.56778592])), (5, array([14.51649982,  4.28089515,  6.52673254])), (6, array([ 0.35862405, 21.15214246,  5.76555905])), (7, array([ 7.09562751, 29.51287575,  5.7683621 ])), (8, array([ 7.0971825 , 21.15450387,  9.878172  ])), (9, array([ 0.35514173, 29.46362353,  9.85603883])), (10, array([14.51867581, 12.58326793, 10.63241789])), (11, array([ 7.78539465,  4.21814917, 10.6044645 ])), (12, array([ 7.77431099, 12.585292  , 14.66831004])), (13, array([14.5266219 ,  4.28426859, 14.66204572])), (14, array([ 0.36025206, 21.20847891, 13.9609839

In [19]:
white_list = ['O', 'H', 'N', 'S', 'Cl']
tolerance_factor_1=0.5
tolerance_factor_2=1.0
max_distance_factor=20.0
relaxed=True
other_metals=['Al', 'In', 'Ga']
my_args = (white_list, tolerance_factor_1, tolerance_factor_2, max_distance_factor, relaxed,other_metals)
#df_all['channel_indices_2'] = df_all.apply(get_channel_indices, args=my_args, axis=1)
a['channel_indices'] = a.apply(get_channel_indices, args=my_args, axis=1)

In [20]:
a['distances'] = a.apply(get_distances, axis=1)
#a['alignments'] = a.apply(get_alignments, axis=1)

[0.0098085  0.00763981 6.48387771]   2
[0.0098085  1.00763981 5.48387771]   2
[1.0098085  0.00763981 5.48387771]   2
[0.51655163 0.61687081 6.09383589]   2
[0.64928558 0.77644997 5.48387771]   2
[0.64883464 0.00763981 6.25306273]   2
[0.0098085  0.71432472 6.19140612]   2
[ 4.36854295  5.56587846 -0.38752416]   2
[ 4.36854295  6.56587846 -1.38752416]   2
[ 5.36854295  5.56587846 -1.38752416]   2
[ 4.87528608  6.17510946 -0.77756598]   2
[ 5.00802003  6.33468861 -1.38752416]   2
[ 4.36854295  6.27256336 -0.67999576]   2

[7.77793995 4.28696734 2.40905736]   1

[11.56293108  3.5636401   2.10201187]   2
[11.6750853   3.70961862  1.52383615]   2
[ 7.09090027 11.25349595  4.1622387 ]   2
[ 1.29560137 13.08211653 11.63095288]   2
[ 1.40775558 13.22809505 11.05277715]   2
[5.69898385 5.40470656 9.63876715]   2
[5.81113806 5.55068507 9.06059143]   2

[4.18334742 1.18916837 5.41617695]   2
[4.18334742 2.18916837 4.41617695]   2
[5.18334742 1.18916837 4.41617695]   2
[4.63370073 1.73003243 5.126

In [ ]:
a

In [ ]:
df_all['distances'] = df_all.apply(get_distances, axis=1)
df_all['alignments'] = df_all.apply(get_alignments, axis=1)

In [ ]:
def get_channel_stats(channels):
    if len(channels) == 0:
        return 0
    elif channels[0][0] == channels[0][1]:
        return 1
    else:
        return 2
    
    
empty = 0
redundant = 0
good = 0
for c in df_all['channel_indices_2'].values:
    res = get_channel_stats(c)
    if res == 0:
        empty += 1
    elif res == 1:
        redundant += 1
    elif res == 2:
        good += 1
        
print(f"out of {len(df_all)} MOFs:")
print(f"{empty} {100*empty/len(df_all)}% are empty")
print(f"{redundant} {100*redundant/len(df_all)}% are periodic")
print(f"{good} {100*good/len(df_all)}% are good")

In [ ]:
picklefile = open('df.pkl', 'wb')
pickle.dump(df_all, picklefile)
picklefile.close()

In [ ]:
# relaxed_dir = "/Users/ben/Documents/EPFL/Research/LSMO/FINAL_PUSH/relaxed_structures/"
relaxed_dir = "/Users/ben/Documents/EPFL/Research/LSMO/FINAL_PUSH/MOF_analysis/cifs/relaxed_structures/"
relaxed_structures = os.listdir(relaxed_dir)
relaxed_structures = [relaxed_dir+cif for cif in relaxed_structures]
relaxed_structures.sort()
print(len(relaxed_structures))



original_dir = "/Users/ben/Documents/EPFL/Research/LSMO/FINAL_PUSH/MOF_analysis/cifs/original_structures/"
original_structures = os.listdir(original_dir)
original_structures = [original_dir+cif for cif in original_structures if cif in os.listdir(relaxed_dir)]
original_structures.sort()
print(len(original_structures))

In [ ]:
num_z = 0
for r_s in relaxed_structures[:20]:
    s = Structure.from_file(r_s)
    ms, non_ms = get_metallic_substructure(s, white_list=['O','H'])#,'S','Cl','N'])
    channels = get_channels(ms, non_ms,
                           tolerance_factor_1=2.0,
                           tolerance_factor_2=2.0,
                            max_distance_factor=5.0)
    if len(channels) == 0:
        num_z += 1
    print(channels)
print(f"zeros: {num_z}")

In [ ]:
num_z = 0
for o_s in original_structures[:20]:
    s = Structure.from_file(o_s)
    ms, non_ms = get_metallic_substructure(s,white_list=['O','H'])#,'S','Cl','N']) 
    channels = get_channels(ms, non_ms,
                           tolerance_factor_1=2.0,
                           tolerance_factor_2=2.0,
                            max_distance_factor=5.0)
    print(channels)
    if len(channels) == 0:
        num_z += 1
        
print(f"zeros: {num_z}")